Run simulation

In [1]:
! lmp < lammps_v3.in

zsh:1: command not found: lmp


Analyze stream

In [3]:
from imdclient.IMDREADER import IMDReader
import MDAnalysis as mda
from MDAnalysis.analysis.rms import RMSF
from MDAnalysis.analysis.rdf import InterRDF
import logging


import concurrent.futures
u = mda.Universe("topology_after_min.data", "imd://localhost:8888", atom_style="id type x y z",)
g = u.atoms

rdf = InterRDF(g, g)
rdf._ts = u.trajectory.ts

rmsf = RMSF(g)
rmsf._ts = u.trajectory.ts

rdf._prepare()
rmsf._prepare()

executor = concurrent.futures.ThreadPoolExecutor()

count = 0
for ts in u.trajectory:
    rdf_frame = executor.submit(rdf._single_frame)

    rmsf._frame_index = count
    rmsf_frame =executor.submit(rmsf._single_frame)

    # Wait for each thread to finish
    rdf_frame.result()
    rmsf_frame.result()

    count += 1

rdf.n_frames = count
rdf._conclude()

rmsf._conclude()

rdf.results["rdf"]
rmsf.results["rmsf"]


/Users/ljwoods2/miniforge3/envs/lammps-demos/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


array([0.08108695, 0.06970932, 0.09123742, ..., 0.0566763 , 0.08996709,
       0.05293365])